Загрузка данных. Размерность (batch_size, max_len, num_features) - (количество таблиц, строк, столбцов)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm

In [5]:
import numpy as np
X = np.load("X.npy")
y = np.load("y.npy")



In [6]:
print(np.isnan(X).any(), np.isnan(y).any()) # булев массив, хотябы 1 true
print(np.isinf(X).any(), np.isinf(y).any())


True False
False False


In [7]:
X = np.nan_to_num(X)
y = np.nan_to_num(y)

In [8]:
print("X min/max:", X.min(), X.max())
print("y min/max:", y.min(), y.max())


X min/max: -17531.0 312972.0
y min/max: 0.0 127.301369863


Есть разброс значений, а нейронки к ним чувствительны, требуется предобработка

In [9]:
from sklearn.model_selection import train_test_split

# отделяем тестовую выборку (20% данных)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# из оставшихся данных выделяем валидационную выборку (25% от оставшихся = 20% от всех данных)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

Масштабирование признаков

In [10]:
from sklearn.preprocessing import StandardScaler

# масштабируем только по обучающей выборке, чтобы не подглядывать в валидацию/тест
scaler = StandardScaler()

# преобразуем каждую части (train/val/test) отдельно по форме (N, T, F) -> (N*T, F)
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

# fit только на тренировочных данных
scaler.fit(X_train_reshaped)

X_train_scaled = scaler.transform(X_train_reshaped).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val_reshaped).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

X_train = X_train_scaled
X_val = X_val_scaled
X_test = X_test_scaled

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset): # свой класс датасета, наследуясь от Dataset
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32) #тензор из нумпай массива
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)
# shuffle перемешивает порядок на каждой эпохе
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) #возвращает батчи в обучении
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [12]:
import torch.nn as nn #нейросетевые слои (например, Linear, LSTM, Conv2d)
# и базовый класс nn.Module для создания собственных моделей

#  Модель (many-to-one). Наследуем от nn и можем использовать встроенные возможности фреймворка 
# (автоматическое вычисление градиентов, перенос на GPU, сохранение модели и т.д.
class RNNModel(nn.Module):  
    #размерность входного вектора, сколько нейронов в скрытом состоянии,колво слоев lstm,
    #рамерность выхода
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)# полносвязный слой в конце

    def forward(self, x):
        out, (h_n, c_n) = self.rnn(x)  
        #out — выходы LSTM на каждом временном шаге;
        #(h_n, c_n) — последние состояния LSTM (скрытое и ячейковое). 
        
        # берём последнее скрытое состояние (many-to-one)
        last_hidden = h_n[-1]# из всех слоёв берётся скрытое состояние последнего слоя LSTM
        #(это представление всей последовательности)
        out = self.fc(last_hidden)#Пропускаем это скрытое состояние через линейный слой, чтобы получить финальный выход
        return out


In [20]:
# Параметры модели 
input_dim = X.shape[2] #размерность признаков
hidden_dim = 64 # размер скрытого состояния, Чем больше — тем выше способность модели запоминать 
#сложные зависимости, но и сложнее обучение.
num_layers = 1 # число слоёв LSTM
output_dim = 1  # выход - возраст

model = RNNModel(input_dim, hidden_dim, num_layers, output_dim)
criterion = nn.L1Loss()  # mae лучше когда есть выбросы
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)



Каждая эпоха — это один полный проход по всем обучающим данным

In [21]:
num_epochs = 40
best_val_loss = float('inf') # начальное значение для лучшей валидационной потери
patience = 5  # количество эпох для early stopping
patience_counter = 0 # 

for epoch in range(num_epochs):
    # Обучение
    model.train()
    train_loss = 0
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()#обнуляем старые градиенты (иначе они будут суммироваться).
        y_pred = model(X_batch).squeeze()#.squeeze() убирает лишние размерности
        loss = criterion(y_pred, y_batch)
        loss.backward()#вычисляем градиенты (все производные) для параметров модели
        optimizer.step()#обновляем веса модели, используя вычисленные градиенты.
        train_loss += loss.item()#сохраняем значение ошибки для статистики.
    train_loss = train_loss/len(train_loader) # средняя ошибка по всем батчам
    
    # Валидация
    model.eval()
    val_loss = 0
    val_preds = []
    val_targets = []
    with torch.no_grad(): # это контекстный менеджер, который отключает подсчёт градиентов
        for X_batch, y_batch in val_loader: # модель перестаёт «учиться» и просто делает прогноз. для ускорения вычислений
            y_pred = model(X_batch).squeeze()
            val_loss += criterion(y_pred, y_batch).item() # накапливаем mseloss
            # собираем для метрик mse и mae (переводим в numpy)
            val_preds.append(y_pred.cpu().numpy())
            val_targets.append(y_batch.cpu().numpy())
    
    val_loss = val_loss/len(val_loader)
    # склеиваем батчи в один массив
    val_preds = np.concatenate(val_preds, axis=0)
    val_targets = np.concatenate(val_targets, axis=0)

    val_mse = mean_squared_error(val_targets, val_preds)
    val_mae = mean_absolute_error(val_targets, val_preds)
        
    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"Train Loss: {train_loss:.4f} | "
        f"Val MSE: {val_mse:.4f} | "
        f"Val MAE: {val_mae:.4f}"
        )
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Сохранение лучшей модели
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping!")
            break

Epoch 1/40: 100%|██████████| 95/95 [00:00<00:00, 194.54it/s]



Epoch [1/40] Train Loss: 39.5918 | Val MSE: 1458.2007 | Val MAE: 35.0560


Epoch 2/40: 100%|██████████| 95/95 [00:00<00:00, 198.24it/s]



Epoch [2/40] Train Loss: 30.9100 | Val MSE: 1016.8001 | Val MAE: 28.1982


Epoch 3/40: 100%|██████████| 95/95 [00:00<00:00, 209.91it/s]



...

Epoch [21/40] Train Loss: 11.7992 | Val MSE: 230.1857 | Val MAE: 12.1082


Epoch 22/40: 100%|██████████| 95/95 [00:00<00:00, 206.25it/s]



Epoch [22/40] Train Loss: 11.8089 | Val MSE: 230.3460 | Val MAE: 12.1149


Epoch 23/40: 100%|██████████| 95/95 [00:00<00:00, 211.72it/s]

Epoch [23/40] Train Loss: 11.7964 | Val MSE: 230.3345 | Val MAE: 12.1144
Early stopping!


Оценка на тесте

In [22]:
# === Оценка на тестовой выборке ===
model.load_state_dict(torch.load("best_model.pth"))# загрузим лучшую модель
model.eval()

test_preds = []
test_targets = []
test_loss = 0.0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch).squeeze()
        test_loss += criterion(y_pred, y_batch).item()
        test_preds.append(y_pred.cpu().numpy())
        test_targets.append(y_batch.cpu().numpy())

test_loss = test_loss / len(test_loader)
test_preds = np.concatenate(test_preds, axis=0)
test_targets = np.concatenate(test_targets, axis=0)

test_mse = mean_squared_error(test_targets, test_preds)
test_mae = mean_absolute_error(test_targets, test_preds)

print(f"Test Loss: {test_loss:.4f} Test MSE: {test_mse:.4f} Test MAE: {test_mae:.4f}")

Test Loss: 12.5393 Test MSE: 239.2224 Test MAE: 12.5884
